# Algebra lineal con NumPy

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Una base de datos es una tabla donde cada fila corresponde a una medición/sujeto/evento y cada columna a un atributo

> Si nuestros atributos son numéricos entonces podemos interpretar la tabla como una matriz y cada ejemplo como un vector 

Considere el siguiente ejemplo de una base de datos que describe el consumo de helados promedio para un día en particular (*cons*) en función del ingreso familiar promedio (*income*), la temperatura promedio (*temp*) y el precio promedio de los helados (*price*)

In [2]:
df = pd.read_csv('../data/helados.csv', index_col=0)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/helados.csv'

De la tabla podemos apreciar que cada medición es un vector de cuatro componentes. Es decir que podemos representar la base de datos como una matriz $A \in \mathbb{R}^{30 \times 4}$

Con esta interpretación de matriz podemos usar las **herramientas de algebra lineal** para analizar estos datos

## ¿Qué es el álgebra lineal?

Es la rama de las matemáticas que analiza los sistemas de ecuaciones lineales de tipo

$$
Ax = b
$$

Es decir donde las incognitas $x$ se relacionan entre si usando sólo **adiciones** y **multiplicaciones**

Si nuestro ejemplos son vectores y nuestra base de datos es una matriz podemos usar herramientas de álgebra lineal para

**1** Medir que tan similares (o distintos) son dos ejemplos: **Distancia vectorial**

$$
d_{ij} = \|\vec z_i - \vec z_j \|
$$

**2** Comprimir nuestra base de datos: **Factorización de matrices**

$$
A = L D L^T
$$

**3** Predecir un atributo en función de otros: **Regresión lineal**

$$
y_i = m x_i + b = \begin{pmatrix} 1 & x_i \end{pmatrix} \begin{pmatrix} b \\ m \end{pmatrix} \quad \forall i
$$

En la presente y siguiente lección nos enfocaremos en la regresión lineal como un caso particular de solución de un sistema de ecuaciones lineales

## Sistemas de ecuaciones lineales


Respecto al ejemplo de los helados, la siguiente es una pregunta interesante

> ¿Es posible predecir el consumo de helados en función del ingreso, la temperatura y el precio?

Para responder esta pregunta podríamos proponer y estudiar un **modelo matemático** de la forma

$$
f(\text{ingreso}, \text{temperatura}, \text{precio}) \rightarrow \text{consumo}
$$

Existen muchas opciones de modelos, sin embargo es conveniente partir con un **modelo simple**


En general el modelo más simple es aquel que es **lineal en sus parámetros**, es decir 

$$
\begin{align}
\text{consumo} &= \text{ingreso} \cdot x + \text{temperatura} \cdot  y + \text{precio} \cdot  z + w\nonumber \\
& = \begin{pmatrix} \text{ingreso} & \text{temperatura} & \text{precio} & 1\end{pmatrix} \begin{pmatrix} x \\ y \\ z \\ w\end{pmatrix}
\end{align}
$$

En este caso **ajustar** el modelo se reduce a encontrar los valores de los **parámetros o incógnitas** $x$, $y$, $z$ y $w$. Cada ejemplo de nuestra base de datos corresponde a una ecuación como la mostrada anteriormente. Es decir que en este caso particular tenemos un sistema lineal con 30 ecuaciones y 4 incógnitas

En el caso más general un **sistema lineal en sus parámetros** con $N$ ecuaciones y $M$ incógnitas se ve así:

$$
\begin{align}
a_{11} x_{1} + a_{12} x_{2} + \ldots + a_{1M} x_M &= b_1 \nonumber \\
a_{21} x_{1} + a_{22} x_{2} + \ldots + a_{2M} x_M &= b_2  \nonumber \\
\vdots  \nonumber \\
a_{N1} x_{1} + a_{N2} x_{2} + \ldots + a_{NM} x_M &= b_N  \nonumber \\
\end{align}
$$

que en representación matricial es

$$
A x = b
$$

El álgebra lineal nos da herramientas para resolver este sistema. Las soluciones dependen de las características del sistema en particular. Veremos tres casos, de los cuales dos son revisados en detalle en esta lección y un tercero se revisará en la lección {ref}`unit2-linear-2`

### Sistema de ecuaciones cuadrado (Caso $N=M$)


Este es un caso particular donde la matriz $A$ tiene igual número de filas y columnas

Estos sistemas pueden resolverse usando inversión de matrices como se muestra a continuación

$$
\begin{align}
A x &=  b \nonumber \\
A^{-1} A x &= A^{-1} b \nonumber \\
x &= A^{-1} b \nonumber 
\end{align}
$$

donde $A A^{-1} = I$, es decir debemos encontrar la inversa de la matriz $A$

Podemos invertir matrices usando la función `inv` del módulo [`linalg`](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html) de NumPy

```python
np.linalg.inv(A # Una matriz cuadrada
             )
```


**Verificar invertibilidad de una matriz**

El sistema tendrá solución siempre y cuando la matriz $A$ sea **no-singular**

Podemos verificar si $A$ es singular comprobando si su determinante es igual a cero con la función `det`

```python
np.linalg.det(A # Una matriz cuadrada
             )
```

Otra forma de verificar si una matriz es invertible es comprobar que todas sus columnas sean linealmente independientes (LI). Esto es equivalente a que su rango sea igual al número de columnas, lo cual se puede verificar con la función `matrix_rank`

```python
np.linalg.matrix_rank(A # Un arreglo multidimensional
                     )
```

**Análisis de errores y *condition number***

Incluso con determinante distinto de cero podríamos no ser capaces de resolver un sistema numéricamente sin errores

Imaginemos una pequeña variación en $b$ denominada $\delta b$. Esta variación provoca a su vez una pequeña variación en $x$ denominada $\delta x$. Se puede encontrar una cota que compara el error relativo de $b$ y $x$ como

$$
\frac{\| \delta x \|}{\|x\|} \leq \frac{\| A^{-1} \|  \|\delta b\|}{\|x\|}  = \|A^{-1}\| \|A\| \frac{\| \delta b \|}{\|b\|} 
$$

donde se usó que $A \delta x = \delta b$ (propiedad de linealidad)

> Esto significa que un pequeño error relativo en $b$ puede causar un gran error en $x$ 

El estimador de $\|A^{-1}\| \|A\|$ se llama *condition number*. Un sistema se dice "bien condicionado" si este valor es cercano a $1$ y "mal condicionado" si es mucho mayor que $1$.

Podemos calcular el *condition number* con la función de NumPy `cond` como se muestra a continuación

```python
np.linalg.cond(x, # Arreglo multidimensional
               p # El orden de la norma: 1, 2, 'fro',...
              )
```

La norma de $A$ denominada $\| A\|$ es 

> una medida del "tamaño" o "magnitud" de $A$ en el espacio

Podemos calcular la norma o tamaño de un vector o matriz usando

```python
np.linalg.norm(x, # Arreglo multidimensional
               ord, # El orden de la norma: 1, 2, 'fro', ...
               axis, # Sobre cual/cualeses eje/s se calcula
               ...
              )
```

Las normas más utilizadas son la

- norma euclidana (`ord=2`) para vectores 
- norma de [Frobenius](https://www.sciencedirect.com/topics/engineering/frobenius-norm) (`ord='fro'`) para matrices


### Resolviendo sistemas cuadrados eficientemente

En general nos interesa sólo $x$ y no $A^{-1}$. Si un sistema de ecuaciones es grande es preferible no calcular la inversa de $A$ por su alto costo computacional. 

Podemos encontrar $x$ directamente en un sistema cuadrado usando la función `solve` 

```python
>>> np.linalg.solve(A, b)
```

Comparemos el resultado obtenido con respecto a calcular la inversa. Para esto podemos usar la función `allclose`

In [ ]:
N = 2000
A = np.random.rand(N, N) # Matriz cuadrada
b = np.random.rand(N, 1) # Vector
np.allclose(np.linalg.solve(A, b), 
            np.dot(np.linalg.inv(A), b))

Es decir que no hay diferencia entre los resultados. Comparemos ahora el tiempo de ejecución 

In [ ]:
%timeit -r5 -n5 np.dot(np.linalg.inv(A), b)
%timeit -r5 -n5 np.linalg.solve(A, b)

Usar `solve` es un poco más de tres veces más veloz que utilizar `inv`+`dot`

¿Cómo puede ser posible esto? 

La respuesta es que `solve` realiza internamente una factorización del tipo

$$
\begin{align}
A x &= b \nonumber \\
LU x &= b \nonumber \\
L z &= b \nonumber
\end{align}
$$

Donde $L$ es una matriz triangular inferior (lower) y $U$ es una matriz triangular superior (upper)

$$
L = \begin{pmatrix} 
l_{11} & 0 & 0 & \ldots & 0 & 0 \\ 
l_{21} & l_{22} & 0 &\ldots & 0 & 0 \\ 
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
l_{N1} & l_{N2} & l_{N3} & \ldots & l_{N(N-1)} & l_{NN} \\ 
\end{pmatrix} \quad
U = \begin{pmatrix} 
u_{11} & u_{11} & u_{13} & \ldots & u_{1(N-1)} & u_{1N} \\ 
u_{21} & u_{22} & u_{32} &\ldots & u_{2(N-1)} & 0 \\ 
\vdots & \vdots & \vdots &\ldots & \ddots & \vdots \\
u_{N1} & 0 & 0 & \ldots & 0 & 0\\ 
\end{pmatrix}
$$

Luego $z$ se puede obtener recursivamente

$$
z_1 = \frac{b_1}{l_{11}}
$$
$$
z_2 = \frac{b_2 - l_{21} z_1}{l_{22}}
$$
$$
z_i = \frac{b_i - \sum_{j=1}^{i-1} l_{ij} z_j}{l_{ii}}
$$

y $x$ se puede obtener recursivamente de $z$

La librería `scipy` nos ofrece en su modulo `linalg` la función `lu` para factorizar

### Ejercicio práctico

Sea el sistema de ecuaciones

$$
\begin{align}
-x_{1} + 5 x_{2} &= 2 \nonumber \\
2 x_{1} + 3 x_{2} &= 1  \nonumber 
\end{align}
$$

1. ¿Cuántas ecuaciones e incognitas tiene este sistema? ¿Es este un sistema cuadrado?
1. Escriba $A$, $b$  y compruebe si $A$ es invertible (determinante y rango)
1. Encuentre la inversa de $A$ y usela para calcular $x$
1. (Extra) Represente geometricamente el sistema y encuentra la solución de forma gráfica

Repita para los sistemas

$$
\begin{align}
x_{1} + 5 x_{2} &= 2 \nonumber \\
2 x_{1} + 10 x_{2} &= 6  \nonumber 
\end{align}
$$

y

$$
\begin{align}
x_{1} + 5 x_{2} &= 2 \nonumber \\
2 x_{1} + 10 x_{2} &= 4  \nonumber 
\end{align}
$$

¿Qué puede decir de estos sistemas?

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('Z47FrSIdIAg')

### Sistema rectangular (caso $N\neq M$)


Consideremos que

- Las incognitas de un sistema representan sus grados de libertad
- Las ecuaciones de un sistema representan sus restricciones

Si tenemos un sistema 

- con más ecuaciones que incognitas: el sistema está sobredeterminado 
- con más incognitas que ecuaciones: el sistema está infradeterminado

y en ambos casos la matriz $A$ ya no es cuadrada, es decir ya no podemos calcular la inversa. Debemos utilizar otros métodos dependiendo del caso

### Sistema sobre-determinado (caso $N>M$)

Definamos el vector de error $e = Ax - b$

Podemos encontrar una solución aproximada minimizando la norma euclidiana del error

$$
\begin{align}
\hat x &= \min_x \|e\|_2^2 \nonumber \\
& = \min_x e^T e \nonumber \\
& = \min_x (Ax -b)^T (Ax -b) \nonumber \\
\end{align}
$$

Lo cual se conoce como el **Problema de mínimos cuadrados**

Para continuar tomamos la última expresión y derivamos con respecto a $x$

$$
\begin{align}
\frac{d}{dx}  (A x - b)^T (A x -b) &= 2 A^T (A x -b) \nonumber \\
&= 2A^T A x - 2A^T b = 0 \nonumber \\
\rightarrow \hat x &= (A^T A)^{-1} A^T b \nonumber \\
&= A^{\dagger} b \nonumber \\
\end{align}
$$

donde $A^{\dagger} = (A^T A)^{-1} A^T$ se conoce como la pseudo-inversa de [Moore-Penrose](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse)

Podemos calcular la pseudo inversa con NumPy usando la función `pinv` como se muestra a continuación

```python
np.linalg.pinv(a, # Arreglo multidimensional
               rcond=1e-15, # Los valores singulares más pequeños que rcond se anulan
               hermitian=False # Booleano para indicar si a tiene simetría hermítica
              )
```

Si sólo queremos obtener la solución $\hat x$ podemos usar 

```python
np.linalg.lstsq(A, # Matriz rectangular de NxM
                b, # vector de largo N
                rcond # idem np.linalg.pinv
               )
                
```

que retorna una tupla con cuatro elementos

- `x`: El resultado buscado
- `residuals`: La norma del error al cuadrado
- `rank`: El rango de $A$
- `s`: Los valores singulares de $A$

Podemos ver la función `lstsq` como una alternativa a `solve` para el caso rectangular sobredeterminado

### Ejercicio práctico

Encuentre los parámetros de la **ecuación de la recta** que ajuste mejor los datos 

$$
\text{consumo} = \theta_1 \cdot \text{temperatura} + \theta_0
$$

- Identifique y construya el vector $b$ y la matriz $A$ ¿Cuánto vale $N$ y $M$?
- ¿Es este un sistema cuadrado o rectangular? ¿ Es sobre o infra-determinado?
- Encuentre $\theta_0$ y $\theta_1$ que minimiza la suma de errores cuadráticos
- Grafique la recta encontrada usando `matplotlib`

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('FNwIqEZtX2A')

## Resumen de la lección

En esta lección hemos aprendido a:

- Reconocer sistemas de ecuaciones lineales cuadrados y rectangulares
- Utilizar herramientas de álgebra lineal implementadas en `NumPy` para resolver los sistemas de ecuaciones